In [1]:
import json
import pandas as pd
import requests
from bs4  import BeautifulSoup

In [2]:
def href(soup):
    # get all href links from one page 
    href=[]
    for i in soup.find_all("a",class_="question-hyperlink",href=True):
        href.append(i['href'])
    return href

In [3]:
# collect all valid herfs
def clean_empty_hrefs(hrefs):
   # remove all empty lists
    list_hrefs=[]
    for i in hrefs:
        if i!=[]:
            list_hrefs.append(i)
    # merge all elemenets in one list
    herfs_list=[]
    for i in list_hrefs:
        for j in i:
            herfs_list.append(j)
    return herfs_list

In [4]:
def add_prefix(herfs_list):
    # rearrage those links who do not have 'https://stackoverflow.com' prefix
    new_href=[]
    prefix='https://stackoverflow.com'
    for h in herfs_list:
        if 'https' not in h:
            m=prefix+h+"answertab=votes#tab-top"
            new_href.append(m)
        else:
            new_href.append(h+"answertab=votes#tab-top")
    return new_href

In [5]:
# obtian one "soup" of one question 
def single_page_scraper(url):
    req=requests.get(url=url)
    soup=BeautifulSoup(req.text,"html.parser")
    return soup

In [6]:
# obtian the question and answer 
def single_page_question_answer(url):
    page=single_page_scraper(url).find_all("div",class_="post-text",itemprop="text")
    question=[i.find("p").get_text()for i in page][0]
    answer=[i.find("p").get_text() for i in page][1]
    
    return question,answer

In [83]:
# Combine all the functions above together 
def questions_answers(start_page,end_page,topic):
    soups=[]
    for page in range(start_page,end_page):
        req=requests.get(url='https://stackoverflow.com/questions/tagged/{}?tab=votes&page={}&pagesize=15'.format(topic,page))
        soup=BeautifulSoup(req.text,"html.parser")
        soups.append(soup)
    
    print("Soups are ready!")
    # obtain all href
    hrefs=[]
    for soup in soups:
        hrefs.append(href(soup))
    

    herfs_list=clean_empty_hrefs(hrefs)
    new_hrefs_list=add_prefix(herfs_list)
    print("All hrefs are ready!")
    print("All most done!")

    quesitons=[]
    answers=[]
    for url in new_hrefs_list:
        try:
            q,a=single_page_question_answer(url)
            quesitons.append(q)
            answers.append(a)
        except:
            pass
        
    print("Done!")  
            
            
    return quesitons,answers


In [93]:
# all questions and answers about python

In [24]:
Questions,Answers=questions_answers(1,500,"python")

Soups are ready!
All hrefs are ready!
All most done!


In [25]:
Questions1,Answers1=questions_answers(501,1000,"python")

Soups are ready!
All hrefs are ready!
All most done!


In [26]:
Questions2,Answers2=questions_answers(1000,2000,"python")

Soups are ready!
All hrefs are ready!
All most done!


In [59]:
# questions and answers about numpy
Questions3,Answers3=questions_answers(1,501,'numpy')

Soups are ready!
All hrefs are ready!
All most done!


In [63]:
# questions and answers about pandas
Questions4,Answers4=questions_answers(1,501,'pandas')

Soups are ready!
All hrefs are ready!
All most done!
Done!


In [84]:
# questions and answers about matplotlib
Questions5,Answers5=questions_answers(1,501,'matplotlib')

Soups are ready!
All hrefs are ready!
All most done!
Done!


In [86]:
# questions and answers about seaborn
Questions6,Answers6=questions_answers(1,360,'seaborn')

Soups are ready!
All hrefs are ready!
All most done!
Done!


In [88]:
# add up all the data we collected above
As=Answers+Answers1+Answers2+Answers3+Answers4+Answers5+Answers6
Qs=Questions+Questions1+Questions2+Questions3+Questions4+Questions5+Questions6

In [89]:
# create a dataframe to store them togehter 
df=pd.DataFrame()
df["question"]=Qs
df["answer"]=As

In [90]:
df.head() # first five rows

,question,answer
0,What is the use of the yield keyword in Python...,"To understand what yield does, you must unders..."
1,"Given the following code, what does the if __n...",Whenever the Python interpreter reads a source...
2,If Python does not have a ternary conditional ...,"Yes, it was added in version 2.5. The expressi..."
3,"In Python, what are metaclasses and what do we...",A metaclass is the class of a class. A class d...
4,How do you call an external command (as if I'd...,Look at the subprocess module in the standard ...


In [37]:
df.question.iloc[-1]

'What is the principle of physics used in this popular stunt?\nInitially, I thought aerodynamics due to an increase in the angle of attack, but its magnitude is not sufficient to balance the whole body and skateboard. Please, can anyone help me to get about it?'

In [30]:
# check the shape 
df.shape

(140630, 2)

In [92]:
# save as csv file
df.to_csv("df_python_stackoverflow.csv")